Финальный проект. Вариант 2. Шутков Д.Н.

1. Парсинг биржевых данных о движении цен на акции Meta, Netflix, Apple, Amazon, Google.

2. Очистка, преобразование данных.

3. Построение на основании этих данных витрин: 
Название акции;
Суммарный объем торгов за последние сутки; Цена акции на момент открытия торгов для данных суток; Цена акции на момент закрытия торгов для данных суток;
Разница(в %)цен с момента открытия до момента закрытия торгов для данных суток; Минимальный временной интервал, на котором был зафиксирован самый крупный объем торгов для данных суток; Минимальный временной интервал, на котором была зафиксирована максимальная цена для данных суток; Минимальный временной интервал, на котором была зафиксирована минимальная цена торгов для данных суток



├── data                       # Архив с файлами исходных данных <br/>
├── data marts                 # Витрины данных в формате parquet<br/>
├── docs                       # Презентация<br/>
├── idef                       # Функциональная модель проекта <br/>
├── src                        # Исходный код программы <br/>
└── README.md                  # Информация о проекте<br/>


#1. ЗАГРУЗКА ДАННЫХ 

Загрузка данных с API сайта https://www.alphavantage.co
Данные загружаются в папку data в виде csv файлов,
именованных по формату "ГГГГ_ММ_ДД_ТИКЕТАКЦИИ"

Параметры для скачивания с API
"""API Parameters
❚ Required: function
The time series of your choice. In this case, function=TIME_SERIES_INTRADAY
❚ Required: symbol
The name of the equity of your choice. For example: symbol=IBM
❚ Required: interval
Time interval between two consecutive data points in the time series. The following values are supported: 1min, 5min, 15min, 30min, 60min
❚ Optional: adjusted
By default, adjusted=true and the output time series is adjusted by historical split and dividend events. Set adjusted=false to query raw (as-traded) intraday values.
❚ Optional: outputsize
By default, outputsize=compact. Strings compact and full are accepted with the following specifications: compact returns only the latest 100 data points in the intraday time series; full returns the full-length intraday time series. The "compact" option is recommended if you would like to reduce the data size of each API call.
❚ Optional: datatype
By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the intraday time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
❚ Required: apikey
Your API key. Claim your free API key here. """

In [ ]:
#Импорт необходимых библиотек
import csv
import requests
import pandas as pd
import os
from os import path
import dateutil
import datetime
import shutil

#ПЕРЕНЕСТИ В ОТДЕЛЬНЫЙ ФАЙЛ ТАК КАК ИСПОЛЬЗУЮТСЯ ЧАСТО
today = datetime.date.today()
previous_date = today - dateutil.relativedelta.relativedelta(days=1)
year = previous_date.year
month = previous_date.month
day = previous_date.day

#Секретный ключ APIKEY берется из отдельного файла с целью защиты информации от несанцкионированного доступа
with open('/content/api-key.txt',mode="r") as file:
    api_key = file.readline().strip()

#список акций с которыми будем работать
stocks_list = ['META','AAPL', 'AMZN', 'NFLX', 'GOOG']

#Проверим что директория для хранения "сырых" данных существует, если нет - создадим
if path.exists('/content/data') == False:
  os.mkdir('/content/data')

#Циклом проходим по списку акций и загружаем "сырые" данные по API
#Данные по каждой акции ложим в папку Data, имена файлов в формате "ГГГГ_ММ_ДД_ТИКЕТАКЦИИ"
for stock_name in stocks_list:
  url_for_downloading = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&'\
  f'symbol={stock_name}&interval=1min&slice=year1month1&apikey={api_key}'
  with requests.Session() as s:
    download = s.get(url_for_downloading)
    decoded_content = download.content.decode('utf-8')
    data_from_csv = csv.reader(decoded_content.splitlines(), delimiter=',')
    temp_data_list = list(data_from_csv)

#Пока с костылями в виде промежуточных df, далее ИСПРАВИТЬ 
    df=pd.DataFrame(temp_data_list,columns=['time','open','high','low','close','volume'])
    df.drop (index=df.index [0], axis= 0 , inplace= True )
    path_for_download_data = '/content/data/'
    file_name = f'{year}_{month}_{day}_{stock_name}.csv'

#Если файлы уже существуют, то удаляем их: вероятно они повреждены, если мы загружаем их снова
    if path.exists(f'{path_for_download_data}{file_name}') == True:
      os.remove(f'{path_for_download_data}{file_name}')
    df.to_csv(f'{path_for_download_data}{file_name}')
    df.head()#ВРЕМЕННО ДЛЯ ПРОСМОТРА НА СТАДИИ РАЗРАБОТКИ

#2. Процесс Data quality

Скопируем данные, которые будем обрабатывать, в промежуточный слой
в нем произведем очистку данных и необходимые преобразования
и выложим в слой витрин готовые данные

In [ ]:
path_for_copy_data = '/content/data_transform/'
#Проверим что директория для хранения промежуточных данных существует, если нет - создадим
if path.exists(path_for_copy_data) == False:
  os.mkdir(path_for_copy_data)

#Циклом копируем файлы в слой их обработки
for stock_name in stocks_list:
  file_name = f'{year}_{month}_{day}_{stock_name}.csv'

#Если файлы уже существуют, то удаляем их: вероятно они повреждены, если мы копируем их снова
  if path.exists(f'{path_for_copy_data}{file_name}') == True:
    os.remove(f'{path_for_copy_data}{file_name}')
  shutil.copy(f'{path_for_download_data}{file_name}',path_for_copy_data)

df=pd.read_csv(f'{path_for_copy_data}{file_name}')
df.head(20)


#3.Построение витрин

    ВЫХОДНЫЕ ДАННЫЕ:
    Суррогатный ключ категории
    Название акции
    Суммарный объем торгов за последние сутки
    Цена акции на момент открытия торгов для данных суток
    Цена акции на момент закрытия торгов для данных суток
    Разница(в %) цен с момента открытия до момента закрытия торгов для данных суток
    Минимальный временной интервал, на котором был зафиксирован самый крупный объем торгов для данных суток
    Минимальный временной интервал, на котором была зафиксирована максимальная цена для данных суток
    Минимальный временной интервал, на котором была зафиксирована минимальная цена торгов для данных суток

In [ ]:
#Предполагается выгрузка данных в Postgres
#далее выгрузка из него витрин в отдельный слой (папка data_marts)

In [ ]:
#ЗАГОТОВКИ ДЛЯ РАБОТЫ С AIRFLOW
# !pip install apache-airflow
# !airflow initdb
# !airflow webserver -p 8080

In [ ]:
#ЗАГОТОВКИ ДЛЯ РАБОТЫ С AIRFLOW
# from datetime import datetime
# from airflow import DAG
# from airflow.operators.bash import BashOperator
# from airflow.operators.python import PythonOperator
# from airflow.models import Variable
# import pandas as pd
# import json

# dag_downloading_args = {
# 	'owner': 'Dmitrii',
# 	'depends_on_past': False,
# 	'start_date': datetime(2022, 12, 30, tz='Europe/Moscow'),
# 	'email': ['shutkov.dmitry2013@yandex.ru'],
# 	'email_on_failure': True,
# 	'email_on_retry': True,
# 	'retries': 3,
# 	'retry_delay': timedelta(minutes=5),
# 	'schedule_interval': '@dayly',
# }  

# dag = DAG(
#     dag_id='stocks_information_dayly',
#     schedule_interval='30 1 * * *',
#     catchup=False,
#     default_args=dag_downloading_args
# )


# # a.Скачайте произвольный csv-файл (ссылка на файл указана в Variables)
# def read_csv():
#     url = Variable.get("url_csv")
#     r = pd.read_csv(url)
#     return r.to_json()

# # В этой функции производится подсчёт строк
# def count_r(**kwargs):
#     json_table = kwargs['ti']
#     r = json_table.xcom_pull(task_ids='read')
#     r = json.loads(r)
#     return len(r["Game Number"])

# #  b. Создайте еще один python-оператор, передайте в него из первого python-оператора количество строк. 
# # Прочитайте файл(с использованием переменных) и добавьте к данному файлу колонку справа, 
# # которая будет номеровать строки в обратном порядке 
# # Сохраните полученный файл.
# def create_column(**kwargs):
#     # Вытаскиваем результаты из предыдущих операторов с помощью Xcom
#     input = kwargs["ti"]
#     table = input.xcom_pull(task_ids = "read")
#     count = input.xcom_pull(task_ids = "count")

#     df = pd.read_json(table)
#     reverse_list = []
#     for i in range(count, 0, -1):
#         reverse_list.append(i)
   
#     # Добавляем столбец в DataFrame, конвертируем в csv и сохраняем.
#     df["reverse"] = reverse_list
#     df.to_csv('files/file.csv')

# with DAG(dag_id="a_start_dag", start_date=datetime(2022, 1, 1), schedule="0 0 * * *") as dag:

#     python_read = PythonOperator(task_id = "read", python_callable = read_csv)
#     python_count = PythonOperator(task_id = "count", python_callable = count_r)
#     python_create_column = PythonOperator(task_id = "create", python_callable = create_column)

#     bash_read = BashOperator(task_id = "bash_cat", bash_command = "mv /opt/airflow/files/file.csv /opt/airflow/finish/")
#     bash_success = BashOperator(task_id = "Success", bash_command = "echo Success")

#     python_read >> python_count >> python_create_column >> bash_read >> bash_success
